實作 **ResNet**，它解決了深層網路中常見的梯度消失和退化問題，並且是業界廣泛使用的網路之一。

使用 **TensorFlow** 實作一個簡化版的 **ResNet** 架構，這個版本使用 ResNet-18 的基本結構。

---

### **實作說明：**

1. **資料集：**
   我們使用了 CIFAR-10 資料集，它包含 10 個類別的彩色圖像，並且經過了標準的正規化處理（將像素值縮放到 [0, 1] 範圍內）。

2. **ResNet 殘差塊：**
   每個殘差塊由兩個卷積層組成，並通過跳躍連接將輸入直接加到輸出上。這樣可以有效解決網路變深時出現的梯度消失問題。

3. **模型結構：**
   - 初始卷積層使用了 $7 \times 7$ 卷積核，步長為 2，後接一個最大池化層。
   - 模型中共有 4 個殘差階段（每階段包含 2 個殘差塊），每個階段的輸出通道數依次增大（64、128、256、512）。
   - 最後使用全局平均池化，將特徵圖展平，並經過一個全連接層輸出分類結果。

4. **訓練與評估：**
   模型使用了 Adam 優化器，並在 10 個 epoch 內訓練，最終評估模型的準確率。

---

### **結論：**

這個 ResNet-18 模型展示了如何實作和使用殘差網路，並且在 CIFAR-10 這樣的圖像分類任務上具有不錯的表現。

ResNet 是一種非常靈活的架構，你可以根據具體任務需求，通過調整網路深度（如 ResNet-50、ResNet-101）來提升性能。

### **ResNet-18 實作（TensorFlow）**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.datasets import cifar10

# 下載並預處理 CIFAR-10 資料集。這裡將圖像數據正規化到 [0, 1] 範圍。
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # 將數據正規化

# 殘差塊定義
# 殘差塊是 ResNet 的核心組件。它通過跳躍連接（shortcut connection）來解決深層網絡中的梯度消失問題。
def residual_block(x, filters, kernel_size=3, stride=1, conv_shortcut=True):
    """ResNet 殘差塊"""
    shortcut = x
    if conv_shortcut:
        # 如果需要捷徑卷積，則對捷徑進行卷積和批量正規化
        shortcut = layers.Conv2D(filters, 1, strides=stride)(x)
        shortcut = layers.BatchNormalization()(shortcut)

    # 第一個卷積層
    x = layers.Conv2D(filters, kernel_size, padding='same', strides=stride)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # 第二個卷積層
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # 將捷徑和卷積層的輸出相加
    x = layers.Add()([shortcut, x])  # 跳躍連接
    x = layers.ReLU()(x)
    return x

# ResNet 模型定義
# 包括初始卷積層、殘差塊和最終的全連接層。
def build_resnet(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # 初始卷積層
    x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # ResNet 殘差塊
    x = residual_block(x, 64, conv_shortcut=False)  # 第一個殘差塊，不使用捷徑卷積
    x = residual_block(x, 64)  # 第二個殘差塊，使用捷徑卷積
    
    x = residual_block(x, 128, stride=2)  # 第三個殘差塊，增加通道數並減少空間維度
    x = residual_block(x, 128)  # 第四個殘差塊

    x = residual_block(x, 256, stride=2)  # 第五個殘差塊
    x = residual_block(x, 256)  # 第六個殘差塊

    x = residual_block(x, 512, stride=2)  # 第七個殘差塊
    x = residual_block(x, 512)  # 第八個殘差塊

    # 全局平均池化層
    x = layers.GlobalAveragePooling2D()(x)

    # 最後的全連接層
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # 建立模型
    model = models.Model(inputs, outputs)
    return model

# 建立 ResNet 模型
resnet_model = build_resnet(input_shape=(32, 32, 3), num_classes=10)
# 編譯模型。使用 adam 優化器和 sparse_categorical_crossentropy 損失函數來編譯模型。
resnet_model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

# 訓練模型
resnet_model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# 評估模型
test_loss, test_acc = resnet_model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')